In [1]:
from langgraph.graph import START,END,StateGraph
from typing import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
import os
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()
import os
api_key=os.getenv("GOOGLE_API_KEY")

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key)

In [4]:
class JokeState(TypedDict):
    """A state for a joke."""
    topic: str
    joke: str
    explaination: str
    

In [5]:
def generate_joke(state: JokeState ):
    # Generate a joke based on the state
    prompt=f'Generate a joke on the topic - {state["topic"]}'
    response=llm.invoke(prompt).content
    return {'joke':response}

In [6]:
def generate_explaination(state: JokeState):
    """Generate an explanation for the current state of the joke."""

    prompt=f'Write an explaination for the joke {state["joke"]}'
    response=llm.invoke(prompt).content
    return {'explaination':response}

In [7]:
graph=StateGraph(JokeState)

## ADD NODE
graph.add_node('generate_joke',generate_joke)
graph.add_node('generate_explaination',generate_explaination)

## ADD EDGE
graph.add_edge(START ,'generate_joke')
graph.add_edge('generate_joke' ,'generate_explaination')
graph.add_edge('generate_explaination',END)

## PERSISTENCE
checkpointer=InMemorySaver()

## COMPILE
workflow=graph.compile(checkpointer=checkpointer)

In [9]:
config_1={'configurable': {'thread_id': '1'}}
workflow.invoke({'topic':'pizza'},config=config_1)

{'topic': 'pizza',
 'joke': "Why did the pizza get a bad reputation?\n\nBecause it was always late, and people just couldn't **crust** it anymore!",
 'explaination': 'This joke is a classic example of a **pun**!\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The pizza is always late, which naturally makes people frustrated and lose faith in its reliability. If something is always late, you can\'t **trust** it.\n\n2.  **The Pun:** The word "crust" (the outer edge of a pizza) sounds almost exactly like the word "trust" (to have confidence in someone or something).\n\n3.  **The Punchline:** The joke replaces "trust" with "crust" because we\'re talking about a pizza. So, "people just couldn\'t **crust** it anymore" is a silly, pizza-themed way of saying they couldn\'t **trust** it anymore to be on time.\n\nThe humor comes from the unexpected, yet perfectly fitting, play on words, connecting the pizza\'s physical characteristic (crust) with the concept of reliability (trust).'}

In [11]:
## Get final state value
workflow.get_state(config_1)

StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza get a bad reputation?\n\nBecause it was always late, and people just couldn't **crust** it anymore!", 'explaination': 'This joke is a classic example of a **pun**!\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The pizza is always late, which naturally makes people frustrated and lose faith in its reliability. If something is always late, you can\'t **trust** it.\n\n2.  **The Pun:** The word "crust" (the outer edge of a pizza) sounds almost exactly like the word "trust" (to have confidence in someone or something).\n\n3.  **The Punchline:** The joke replaces "trust" with "crust" because we\'re talking about a pizza. So, "people just couldn\'t **crust** it anymore" is a silly, pizza-themed way of saying they couldn\'t **trust** it anymore to be on time.\n\nThe humor comes from the unexpected, yet perfectly fitting, play on words, connecting the pizza\'s physical characteristic (crust) with the concept of reliability (tr

In [13]:
## Get Entire state value
list(workflow.get_state_history(config_1))

[StateSnapshot(values={'topic': 'pizza', 'joke': "Why did the pizza get a bad reputation?\n\nBecause it was always late, and people just couldn't **crust** it anymore!", 'explaination': 'This joke is a classic example of a **pun**!\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The pizza is always late, which naturally makes people frustrated and lose faith in its reliability. If something is always late, you can\'t **trust** it.\n\n2.  **The Pun:** The word "crust" (the outer edge of a pizza) sounds almost exactly like the word "trust" (to have confidence in someone or something).\n\n3.  **The Punchline:** The joke replaces "trust" with "crust" because we\'re talking about a pizza. So, "people just couldn\'t **crust** it anymore" is a silly, pizza-themed way of saying they couldn\'t **trust** it anymore to be on time.\n\nThe humor comes from the unexpected, yet perfectly fitting, play on words, connecting the pizza\'s physical characteristic (crust) with the concept of reliability (t

In [14]:
config_2={'configurable': {'thread_id': '2'}}
workflow.invoke({'topic':'Pasta'},config=config_2)

{'topic': 'Pasta',
 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was always thinking about the **past-a**!',
 'explaination': 'This joke is a classic example of a **pun**! It works by playing on the similar sound of two different phrases.\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The joke sets up a scenario where pasta and sauce are personified, acting like humans in a relationship. This makes the idea of them "breaking up" relatable, even though they\'re just food.\n\n2.  **The Punchline:** "Because it was always thinking about the **past-a**!"\n    *   **Sound-alike:** The phrase "past-a" is spelled to look like "pasta" (the food), but it\'s pronounced exactly like "the past."\n    *   **The "Past" Meaning:** In human relationships, if someone is "always thinking about the past," it usually implies they are dwelling on old issues, previous relationships, or a time gone by. This often prevents them from moving forward in the current relationship and can be a

In [15]:
## Get final state value
workflow.get_state(config_2)

StateSnapshot(values={'topic': 'Pasta', 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was always thinking about the **past-a**!', 'explaination': 'This joke is a classic example of a **pun**! It works by playing on the similar sound of two different phrases.\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The joke sets up a scenario where pasta and sauce are personified, acting like humans in a relationship. This makes the idea of them "breaking up" relatable, even though they\'re just food.\n\n2.  **The Punchline:** "Because it was always thinking about the **past-a**!"\n    *   **Sound-alike:** The phrase "past-a" is spelled to look like "pasta" (the food), but it\'s pronounced exactly like "the past."\n    *   **The "Past" Meaning:** In human relationships, if someone is "always thinking about the past," it usually implies they are dwelling on old issues, previous relationships, or a time gone by. This often prevents them from moving forward in the current relati

In [16]:
## Get Entire state value
list(workflow.get_state_history(config_2))

[StateSnapshot(values={'topic': 'Pasta', 'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was always thinking about the **past-a**!', 'explaination': 'This joke is a classic example of a **pun**! It works by playing on the similar sound of two different phrases.\n\nHere\'s the breakdown:\n\n1.  **The Setup:** The joke sets up a scenario where pasta and sauce are personified, acting like humans in a relationship. This makes the idea of them "breaking up" relatable, even though they\'re just food.\n\n2.  **The Punchline:** "Because it was always thinking about the **past-a**!"\n    *   **Sound-alike:** The phrase "past-a" is spelled to look like "pasta" (the food), but it\'s pronounced exactly like "the past."\n    *   **The "Past" Meaning:** In human relationships, if someone is "always thinking about the past," it usually implies they are dwelling on old issues, previous relationships, or a time gone by. This often prevents them from moving forward in the current relat

## Time Travel

In [18]:
workflow.get_state({"configurable": {"thread_id": "2", "checkpoint_id": "1f06ddd1-37fa-69b6-8000-52794171a131"}})

StateSnapshot(values={'topic': 'Pasta'}, next=('generate_joke',), config={'configurable': {'thread_id': '2', 'checkpoint_id': '1f06ddd1-37fa-69b6-8000-52794171a131'}}, metadata={'source': 'loop', 'writes': None, 'step': 0, 'parents': {}, 'thread_id': '2'}, created_at='2025-07-31T07:07:53.081899+00:00', parent_config={'configurable': {'thread_id': '2', 'checkpoint_ns': '', 'checkpoint_id': '1f06ddd1-37f7-69b8-bfff-54e343a406f1'}}, tasks=(PregelTask(id='b43ee502-6a7c-f5ec-3c94-aa8b3c228202', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the pasta break up with the sauce?\n\nBecause it was always thinking about the **past-a**!'}),), interrupts=())

In [20]:
workflow.invoke(None,{"configurable": {"thread_id": "2", "checkpoint_id": "1f06ddd1-37fa-69b6-8000-52794171a131"}})

{'topic': 'Pasta',
 'joke': "Why did the pasta break up with the tomato sauce?\n\nBecause it couldn't *marinara* the relationship anymore!",
 'explaination': 'This joke is a classic pun! Here\'s the breakdown:\n\n1.  **The Setup:** "Why did the pasta break up with the tomato sauce?" This sets up a scenario where two things that usually go together (pasta and sauce) are having relationship problems, treating them like people.\n\n2.  **The Pun:** The humor comes from the word "**marinara**."\n    *   **Literal Meaning:** Marinara is a type of tomato sauce, often served with pasta. This connects directly to the "tomato sauce" in the joke.\n    *   **Sound-Alike:** "Marinara" sounds very similar to "manage" or "maintain" (especially "marry \'n\' era" or "manage \'n\' era").\n\n3.  **The Punchline:** "Because it couldn\'t *marinara* the relationship anymore!"\n    *   When you substitute the sound-alike, it becomes: "Because it couldn\'t **manage/maintain** the relationship anymore!"\n\nThe

In [21]:
list(workflow.get_state_history(config_2))

[StateSnapshot(values={'topic': 'Pasta', 'joke': "Why did the pasta break up with the tomato sauce?\n\nBecause it couldn't *marinara* the relationship anymore!", 'explaination': 'This joke is a classic pun! Here\'s the breakdown:\n\n1.  **The Setup:** "Why did the pasta break up with the tomato sauce?" This sets up a scenario where two things that usually go together (pasta and sauce) are having relationship problems, treating them like people.\n\n2.  **The Pun:** The humor comes from the word "**marinara**."\n    *   **Literal Meaning:** Marinara is a type of tomato sauce, often served with pasta. This connects directly to the "tomato sauce" in the joke.\n    *   **Sound-Alike:** "Marinara" sounds very similar to "manage" or "maintain" (especially "marry \'n\' era" or "manage \'n\' era").\n\n3.  **The Punchline:** "Because it couldn\'t *marinara* the relationship anymore!"\n    *   When you substitute the sound-alike, it becomes: "Because it couldn\'t **manage/maintain** the relations